<a href="https://colab.research.google.com/github/AlokDhanush/RAG_chatbot/blob/main/RAG_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q faiss-cpu sentence-transformers transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.9 MB/s eta 0:00:00


In [2]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

In [5]:
file_path = "/content/rcb_data.txt"
with open(file_path, "r", encoding="utf-8") as f:
    text = f.read()

In [6]:
def chunk_text(text, max_words=100):
    words = text.split()
    return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

chunks = chunk_text(text)

In [7]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-mpnet-base-v2")
embeddings = embedder.encode(chunks, convert_to_numpy=True).astype("float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Device set to use cuda:0


In [9]:
def get_top_chunks(query, k=3):
    query_vec = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, k)
    return [chunks[i] for i in indices[0]]

def build_prompt(context_chunks, query):
    context = "\n".join(context_chunks)
    prompt = f"""[INST] You are a cricket expert assistant. Use only the context to answer.
If the answer is not found in the context, say "I don't know".

### Context:
{context}

### Question:
{query}

### Answer:
[/INST]"""
    return prompt

In [10]:
def ask(query):
    context_chunks = get_top_chunks(query)
    prompt = build_prompt(context_chunks, query)

    result = llm(
        prompt,
        max_new_tokens=200,
        temperature=0.3,
        do_sample=False,
        return_full_text=False
    )

    return "Answer: "+ result[0]['generated_text'].strip()


In [20]:
query = "Who has the most runs for RCB in IPL?"
print("Retrieved Context:\n", "\n---\n".join(get_top_chunks(query)))
ask(query)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Retrieved Context:
 as head coach.[123] RCB qualified for the playoffs by finishing third in the points table but failed to lift the trophy once again, losing to fourth placed Kolkata Knight Riders in the Eliminator.[124][125] Harshal Patel finished the season with 32 wickets, equalling Dwayne Bravo's record for the highest number of wickets taken in a season and winning the Purple Cap.[126][127] Glenn Maxwell was the highest run-scorer for the team with 513 runs, the fifth highest in the season.[128] Virat Kohli became the first-ever batsman to score 6,000 runs in the IPL during this season.[129] This season marked AB de Villiers's
---
for just 49 runs, marking the lowest score ever in an IPL match.[94][95] Virat Kohli was the leading run-scorer for the team with 308 runs in the tournament, while Pawan Negi took the most wickets, claiming 16.[96][97] Ahead of the 2018 IPL, RCB retained Virat Kohli, AB de Villiers, and Sarfaraz Khan.[98] During the auction, they acquired notable player

'Answer: Virat Kohli with 308 runs in the tournament.'

In [12]:
ask("How many runs did Virat Kohli scored in the final match between RCB vs PBKS?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know. The context only mentions Virat Kohli scoring 75 runs in one match, but it does not specify the opponent or the final match."

In [13]:
ask("How many times has RCB played playoff matches?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know."

In [14]:
ask("Who is the owner of RCB?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know. The context does not mention the owner of RCB."

In [15]:
ask("RCB stands for?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Answer: Royal Challengers Bangalore, commonly known as RCB, is a professional T20 franchise cricket team based in Bengaluru, Karnataka, that competes in the Indian Premier League.'

In [16]:
ask("When did RCB win their first title?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Answer: RCB won their first title in 2025.'

In [17]:
ask("Who is the top scorer for RCB in 2025?")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"Answer: I don't know. The context only provides information about the 2023 season, not the 2025 season."